## Reprojecting the Raster

The newly merged raster is in EPSG: 36423 which is a projected coordinate system but the raster needs to be in a geographic coordinate system. Thus it will be reprojected to EPSG 4326.

### Import the Libraries

In [ ]:
import rasterio
import os
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling

### Set the Search Paths

The folder where all the files are located:

In [ ]:
inpath = r"/Users/Jenny/Downloads/Imagery/Images/"

### Reprojecting

In [ ]:
# A funciton that reprojects the rasters to 4326
def reproject_rast(outpath, inpath):
    
    # Desired CRS
    dst_crs = 'EPSG:4326'
    
    # Opens the raster to be reprojected
    with rasterio.open(inpath) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        
        # Reprojects the raster
        with rasterio.open(outpath, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest)

# Function to search through folders to select files to be reprojected
def compare():
    ending_list = []
    
    # Searches for files that end with merge.tif and sets them to a new list
    for root, dirs, files in os.walk(inpath):
        for filename in files:
            if filename.endswith("merge.tif"):
                print('File name is: {0}'.format(filename))
                ending_list.append(filename.split('_',1)[0])
    
    # Creates a unique list from above
    unique_ending = list(set(ending_list))

    # Searches through the unique list to select files that are unique and contain the words merge
    for unique in unique_ending:
        for root, dirs, files in os.walk(inpath):
            for filename in files:
                if filename.startswith(unique):
                    files_unique = [x for x in files if x.startswith(unique)]
                    files_path = [x for x in files_unique if x.endswith("merge.tif")]
                    outpath = inpath + unique + "_reproj.tif"
                    if unique != False:
                        os.chdir(inpath)
                        reproject_rast(outpath, files_path[0])
                else:
                    print("N/A")

compare()